<a href="https://colab.research.google.com/github/shanboii/AISOME23/blob/main/roberta_aisome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip list | grep -E 'transformers|tensorflow'


tensorflow                       2.12.0
tensorflow-datasets              4.9.2
tensorflow-estimator             2.12.0
tensorflow-gcs-config            2.12.0
tensorflow-hub                   0.14.0
tensorflow-io-gcs-filesystem     0.33.0
tensorflow-metadata              1.13.1
tensorflow-probability           0.20.1


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
pip install --upgrade transformers tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 41.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Succes

In [ ]:
!pip uninstall transformers


Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.31.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.31.0


In [ ]:
!pip install transformers

  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)


In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score
import pickle
from tqdm import tqdm, trange
from ast import literal_eval

In [ ]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

**dataset**

In [ ]:
df = pd.read_csv('/content/dataset_final.csv') #-comment-classification-challenge
df.head()

,tweetID,tweet_text,mandatory,country,conspiracy,unnecessary,political,ingredients,side-effect,pharma,none,ineffective,rushed,religious
0,1296010336907038720t,@cath__kath AstraZeneca is made with the kidne...,0,0,0,0,0,1,0,0,0,0,0,0
1,1336808189677940736t,It begins. Please find safe alternatives to th...,0,0,0,0,0,0,1,0,0,0,0,0
2,1329488407307956231t,"@PaolaQP1231 Well, I mean congratulations Covi...",0,0,0,0,0,0,1,0,0,0,0,0
3,1364194604459900934t,@BorisJohnson for those of us that do not wish...,1,0,0,0,0,0,0,0,0,0,0,0
4,1375938799247765515t,She has been trying to speak out: writing lett...,0,0,0,0,0,0,1,0,0,0,1,0


In [ ]:
print('Unique comments: ', df.tweet_text.nunique() == df.shape[0])
print('Null values: ', df.isnull().values.any())
df[df.isna().any(axis=1)]

Unique comments:  True
Null values:  False


,tweetID,tweet_text,mandatory,country,conspiracy,unnecessary,political,ingredients,side-effect,pharma,none,ineffective,rushed,religious


In [ ]:
print('average sentence length: ', df.tweet_text.str.split().str.len().mean())
print('stdev sentence length: ', df.tweet_text.str.split().str.len().std())

average sentence length:  34.68753149884084
stdev sentence length:  12.392942139932323


In [ ]:
cols = df.columns
label_cols = list(cols[2:])
num_labels = len(label_cols)
print('Label columns: ', label_cols)

Label columns:  ['mandatory', 'country', 'conspiracy', 'unnecessary', 'political', 'ingredients', 'side-effect', 'pharma', 'none', 'ineffective', 'rushed', 'religious']


In [ ]:
print('Count of 1 per label: \n', df[label_cols].sum(), '\n') # Label counts, may need to downsample or upsample
print('Count of 0 per label: \n', df[label_cols].eq(0).sum())

Count of 1 per label: 
 mandatory       783
country         201
conspiracy      487
unnecessary     722
political       626
ingredients     436
side-effect    3805
pharma         1273
none            629
ineffective    1672
rushed         1476
religious        64
dtype: int64 

Count of 0 per label: 
 mandatory      9138
country        9720
conspiracy     9434
unnecessary    9199
political      9295
ingredients    9485
side-effect    6116
pharma         8648
none           9292
ineffective    8249
rushed         8445
religious      9857
dtype: int64


In [ ]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle rows

In [ ]:
df['one_hot_labels'] = list(df[label_cols].values)
df.head()

,tweetID,tweet_text,mandatory,country,conspiracy,unnecessary,political,ingredients,side-effect,pharma,none,ineffective,rushed,religious,one_hot_labels
0,1376557786151198725t,Any politician pushing something as unconstitu...,1,0,0,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1329877849629716483t,@DrLaPook @WCCO @pfizer @CBSNews Pfizer by law...,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,1287769310774009863t,@drmattglass @Project_Veritas @delbigtree Yes!...,0,0,0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
3,1371050337281396736t,It's all coming out into the open. A vaccine r...,0,0,0,0,0,0,1,0,0,0,1,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
4,1364260391111557121t,@GovMikeDeWine The only fact is that you're pu...,0,0,0,1,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
labels = list(df.one_hot_labels.values)
comments = list(df.tweet_text.values)

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

In [ ]:
max_length = 100
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


tokenizer outputs:  dict_keys(['input_ids', 'attention_mask'])


In [ ]:
input_ids = encodings['input_ids'] # tokenized and encoded sentences
attention_masks = encodings['attention_mask'] # attention masks

In [ ]:
# Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
label_counts = df.one_hot_labels.astype(str).value_counts()
one_freq = label_counts[label_counts==1].keys()
one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
print('df label indices with only one instance: ', one_freq_idxs)

df label indices with only one instance:  [9629, 8564, 8390, 8385, 8196, 8095, 7504, 7394, 7350, 7042, 6588, 6397, 6173, 6115, 5642, 5167, 5130, 4967, 4616, 4491, 3750, 3560, 2855, 2789, 2474, 1427, 834, 422, 409, 400]


In [ ]:
# Gathering single instance inputs to force into the training set after stratified split
one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
one_freq_labels = [labels.pop(i) for i in one_freq_idxs]

In [ ]:

from sklearn.model_selection import train_test_split

# Split the data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels, train_masks, validation_masks = train_test_split(
    input_ids, labels, attention_masks,
    random_state=2020, test_size=0.10, stratify=labels
)

# Add one frequency data to train data
train_inputs.extend(one_freq_input_ids)
train_labels.extend(one_freq_labels)
train_masks.extend(one_freq_attention_masks)


# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

<ipython-input-24-38edd5c5535f>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_labels = torch.tensor(train_labels)


In [ ]:
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
torch.save(validation_dataloader,'validation_data_loader')
torch.save(train_dataloader,'train_data_loader')

model


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Initialize the RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load model, the pretrained model will include a single linear classification layer on top for classification.
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
model.cuda()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
# setting custom optimization parameters. You may implement a scheduler here as well.
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
from transformers import AdamW


In [ ]:
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)
# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


train model


In [ ]:
import torch

# Check if GPU is available, and if not, use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Your code follows here...


In [ ]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):

  # Training

  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss()
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss()
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

###############################################################################

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy
  threshold = 0.50
  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]
  val_f1_accuracy = f1_score(true_bools,pred_bools,average='macro')*100
  val_flat_accuracy = accuracy_score(true_bools, pred_bools)*100

  print('F1 Validation Accuracy: ', val_f1_accuracy)
  print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 0.10988061000992144


Epoch:  33%|███▎      | 1/3 [02:39<05:19, 159.75s/it]

F1 Validation Accuracy:  61.896621016817065
Flat Validation Accuracy:  59.79797979797979
Train loss: 0.09019458136920418


Epoch:  67%|██████▋   | 2/3 [05:18<02:39, 159.43s/it]

F1 Validation Accuracy:  60.96282105358248
Flat Validation Accuracy:  57.67676767676768
Train loss: 0.07588384747505188


Epoch: 100%|██████████| 3/3 [07:58<00:00, 159.45s/it]

F1 Validation Accuracy:  61.99699308195622
Flat Validation Accuracy:  59.79797979797979


In [ ]:
torch.save(model.state_dict(), 'bert_model_toxic')

loAD PREPROCESS TEST DATA


In [ ]:
test_df = pd.read_csv('/content/test_dataset1.csv')
test_label_cols = list(test_df.columns[3:])
print('Null values: ', test_df.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
test_df.head()

Null values:  False
Same columns between train and test:  True


,tweetID,tweet_text,preprocessed_text,mandatory,country,conspiracy,unnecessary,political,ingredients,side-effect,pharma,none,ineffective,rushed,religious
0,1337160614884552705t,Did you know that the new Pfizer COVID-19 vacc...,know new pfizer covid vaccine chance paralyzed,0,0,0,0,0,0,1,0,0,0,0,0
1,1355176268879306752t,@BBCBreaking 'Vaccinating' young people <40 (w...,vaccinating young people risk covid exposure m...,0,0,0,1,0,0,0,1,0,0,0,0
2,1287711540343431168t,@outsider63 @gemmaod1 This is about been force...,forced take hurried barely tested vaccine ther...,1,0,0,0,0,0,0,0,0,0,1,0
3,1417661084715032578t,Search Results from the #VAERS Database 31y♂️#...,search result database administered,0,0,0,0,0,0,1,0,0,0,0,0
4,1337183965917097984t,@Moondoggie2020 @melanielynngrif @ABC They tes...,tested kid trial daughter shes invited partici...,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()

,tweetID,tweet_text,preprocessed_text,mandatory,country,conspiracy,unnecessary,political,ingredients,side-effect,pharma,none,ineffective,rushed,religious,one_hot_labels
0,1337160614884552705t,Did you know that the new Pfizer COVID-19 vacc...,know new pfizer covid vaccine chance paralyzed,0,0,0,0,0,0,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1,1355176268879306752t,@BBCBreaking 'Vaccinating' young people <40 (w...,vaccinating young people risk covid exposure m...,0,0,0,1,0,0,0,1,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"
2,1287711540343431168t,@outsider63 @gemmaod1 This is about been force...,forced take hurried barely tested vaccine ther...,1,0,0,0,0,0,0,0,0,0,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
3,1417661084715032578t,Search Results from the #VAERS Database 31y♂️#...,search result database administered,0,0,0,0,0,0,1,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
4,1337183965917097984t,@Moondoggie2020 @melanielynngrif @ABC They tes...,tested kid trial daughter shes invited partici...,0,0,0,1,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.preprocessed_text.values)

In [ ]:
#Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']

In [ ]:
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')

<ipython-input-44-6d9346621ae3>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)


prediction

In [ ]:
# Test
# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

# Predict
for i, batch in enumerate(test_dataloader):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids,attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)

    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]
true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

In [ ]:
pred_bools = [pl>0.50 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)
pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

Test F1 Accuracy:  0.6857517482517482
Test Flat Accuracy:  0.546095717884131 

              precision    recall  f1-score   support

   mandatory       0.69      0.68      0.68       155
     country       0.72      0.46      0.56        39
  conspiracy       0.60      0.27      0.38        91
 unnecessary       0.50      0.52      0.51       160
   political       0.67      0.31      0.42       110
 ingredients       0.81      0.45      0.58       114
 side-effect       0.82      0.84      0.83       764
      pharma       0.72      0.54      0.62       258
        none       0.73      0.41      0.53       115
 ineffective       0.71      0.67      0.69       342
      rushed       0.76      0.65      0.71       298
   religious       1.00      0.08      0.14        13

   micro avg       0.74      0.64      0.69      2459
   macro avg       0.73      0.49      0.55      2459
weighted avg       0.74      0.64      0.67      2459
 samples avg       0.70      0.67      0.67      2459



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


OUTPUT

In [ ]:
idx2label = dict(zip(range(12),label_cols))
print(idx2label)



{0: 'mandatory', 1: 'country', 2: 'conspiracy', 3: 'unnecessary', 4: 'political', 5: 'ingredients', 6: 'side-effect', 7: 'pharma', 8: 'none', 9: 'ineffective', 10: 'rushed', 11: 'religious'}


In [ ]:
# Getting indices of where boolean one hot vector true_bools is True so we can use idx2label to gather label names
true_label_idxs, pred_label_idxs=[],[]
for vals in true_bools:
  true_label_idxs.append(np.where(vals)[0].flatten().tolist())
for vals in pred_bools:
  pred_label_idxs.append(np.where(vals)[0].flatten().tolist())

In [ ]:
# Gathering vectors of label names using idx2label
true_label_texts, pred_label_texts = [], []
for vals in true_label_idxs:
  if vals:
    true_label_texts.append([idx2label[val] for val in vals])
  else:
    true_label_texts.append(vals)

for vals in pred_label_idxs:
  if vals:
    pred_label_texts.append([idx2label[val] for val in vals])
  else:
    pred_label_texts.append(vals)

In [ ]:
# Decoding input ids to comment text
preprocessed_texts = [tokenizer.decode(text,skip_special_tokens=True,clean_up_tokenization_spaces=False) for text in tokenized_texts]

In [ ]:
# Converting lists to df
comparisons_df = pd.DataFrame({'preprocessed_text': preprocessed_texts, 'true_labels': true_label_texts, 'pred_labels':pred_label_texts})
comparisons_df.to_csv('comparisons.csv')
comparisons_df.head()

,preprocessed_text,true_labels,pred_labels
0,know new pfizer covid vaccine chance paralyzed,[side-effect],[side-effect]
1,vaccinating young people risk covid exposure m...,"[unnecessary, pharma]",[pharma]
2,forced take hurried barely tested vaccine ther...,"[mandatory, rushed]",[rushed]
3,search result database administered,[side-effect],[]
4,tested kid trial daughter shes invited partici...,[unnecessary],[rushed]


In [ ]:
# Calculate Accuracy - maximize F1 accuracy by tuning threshold values. First with 'macro_thresholds' on the order of e^-1 then with 'micro_thresholds' on the order of e^-2

macro_thresholds = np.array(range(1,10))/10

f1_results, flat_acc_results = [], []
for th in macro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='micro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_macro_th = macro_thresholds[np.argmax(f1_results)] #best macro threshold value

micro_thresholds = (np.array(range(10))/100)+best_macro_th #calculating micro threshold values

f1_results, flat_acc_results = [], []
for th in micro_thresholds:
  pred_bools = [pl>th for pl in pred_labels]
  test_f1_accuracy = f1_score(true_bools,pred_bools,average='macro')
  test_flat_accuracy = accuracy_score(true_bools, pred_bools)
  f1_results.append(test_f1_accuracy)
  flat_acc_results.append(test_flat_accuracy)

best_f1_idx = np.argmax(f1_results) #best threshold value

# Printing and saving classification report
print('Best Threshold: ', micro_thresholds[best_f1_idx])
print('Test F1 Accuracy: ', f1_results[best_f1_idx])
print('Test Flat Accuracy: ', flat_acc_results[best_f1_idx], '\n')

best_pred_bools = [pl>micro_thresholds[best_f1_idx] for pl in pred_labels]
clf_report_optimized = classification_report(true_bools,best_pred_bools, target_names=label_cols)
pickle.dump(clf_report_optimized, open('classification_report_optimized.txt','wb'))
print(clf_report_optimized)

Best Threshold:  0.32
Test F1 Accuracy:  0.5409487140030634
Test Flat Accuracy:  0.506953223767383 

              precision    recall  f1-score   support

   mandatory       0.60      0.79      0.68        61
     country       0.48      0.79      0.59        14
  conspiracy       0.53      0.42      0.47        38
 unnecessary       0.42      0.51      0.46        53
   political       0.64      0.41      0.50        51
 ingredients       0.71      0.55      0.62        31
 side-effect       0.80      0.81      0.81       303
      pharma       0.62      0.67      0.64       100
        none       0.46      0.35      0.40        52
 ineffective       0.59      0.71      0.65       132
      rushed       0.62      0.74      0.67       116
   religious       0.00      0.00      0.00         4

   micro avg       0.65      0.68      0.66       955
   macro avg       0.54      0.56      0.54       955
weighted avg       0.65      0.68      0.66       955
 samples avg       0.66      0.69

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
